<a href="https://colab.research.google.com/github/sak1b0/Thesis/blob/master/lets_get_it_done.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
#import pylab as pl
import pandas as pd
import numpy as np

url='https://raw.githubusercontent.com/sak1b0/Thesis/master/allCSV/nursery.csv'

df=pd.read_csv(url,header=None)
#print(df.iloc[0,:])


#----------to handle nominal values---------

my_arr=[]

#track of column indices 
index=0
for item in df.dtypes:
  if(item=='object'):
    #if data type is nominal adding to the array
    my_arr.append(index)
  index=index+1
  
print('object data types: ',len(my_arr))

df=df.values

if(len(my_arr)>0):
  lbl=LabelEncoder()
  for item in my_arr:
    df[:,item] = lbl.fit_transform(df[:,item])
    
  print('after the conversion: ',df[0])    
#--------------------------------------------


df=np.asarray(df)

X, y = np.split(df,[-1],axis=1)
#issue here !!!!!!!!!!!!!!!!!!!!!!!!!!!!!
y=y.astype('float64')

x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.005,random_state=101)

#to check additional info 
debug=0

def debug_me():
  print('----------debug start----------')
  print('main data: ',df.shape)
  print('x: ',X.shape)
  print('y: ',y.shape)
  print('x_train: ',x_train.shape)
  print('x_test: ',x_test.shape)
  print('y_train: ',y_train.shape)
  print('y_test: ',y_test.shape)
  print('----------debug end----------')



X=np.asarray(X)
y=np.asarray(y)
# y=y.ravel() #vector to array

x_train=np.asarray(x_train)
x_test=np.asarray(x_test)
y_train=np.asarray(y_train)
y_test=np.asarray(y_test)

y_train=y_train.ravel()
y_test=y_test.ravel()

debug_me()
#==========preprocessing done============


#==========optimal number of clusters====


no_of_features=x_train.shape[1]
optimal_k=len(np.unique(y_train))
#print('no of features: ',no_of_features)
#print('actual no of classes: ',optimal_k)
#plt.figure(figsize=(6, 4))
#from sklearn.cluster import KMeans
#wcss = []
#for i in range(1, no_of_features):
#    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 101)
#    kmeans.fit(x_train)
#    wcss.append(kmeans.inertia_)
#plt.plot(range(1, no_of_features), wcss)
#plt.title('The Elbow Method')
#plt.xlabel('Number of clusters')
#plt.ylabel('WCSS')
#plt.show()


#=================experiment===========
print("optimal no. of k = ",optimal_k,'\n')

kmeans = KMeans(n_clusters=optimal_k, init = 'k-means++', random_state= 101)

kmeans.fit(x_train)

#print(kmeans.cluster_centers_)  

print('cluster labels: ',kmeans.labels_[0:5,],'\n')

#creating an array as the index to add column to the main df
ind_arr=np.arange(len(x_train))

#adding the index column to the data so that i can understand later
x_train=np.hstack((ind_arr[:, np.newaxis], x_train))

#to see if it has worked and index column is there or not
#print(pd.DataFrame(x_train[:5,]),'\n')
##print(x_train[:5,],'\n')

##print('-- first column is index --')
#print(pd.DataFrame(x_train[:5,]),'\n')
##print(x_train[:5,],'\n')

#print(pd.DataFrame(x_train[:5,1:-1]),'\n')
##print(x_train[:5,1:-1],'\n') #without the index here

#mergning y(actual labels with x again)
##print(len(x_train))
##print(len(y_train))
#np arrays have to be in the proper shape to be appended
y_train = y_train.reshape((y_train.shape[0], 1))
##print(x_train.shape)
##print(y_train.shape)
x_train=np.append(x_train,y_train, axis=1)
##print(x_train[:5,])

##print(kmeans.labels_.shape)
cluster_labels = np.asarray(kmeans.labels_)
cluster_labels = cluster_labels.reshape((cluster_labels.shape[0], 1))
##print(cluster_labels.shape)

#adding the cluster label column to the data in the last column
train_data=np.append(x_train,cluster_labels, axis=1)
#print('basic train data',train_data[:5])

############### working in office lol #####################
print('my train_data_size: ',train_data.shape)
cluster_number, occur_count = np.unique(train_data[:,-1], return_counts=True)
 
print("Cluster No: " , cluster_number)
print("Instance Count : ", occur_count)

#for i in range(len(cluster_number)):
  #print('Cluster: '+str(cluster_number[i])+' :'+str(occur_count[i]))
  
#should not miss these 2 lines and not in the loop
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)
y_test = y_test.ravel()
 
model = GaussianNB()

#all the predictions of different models will be saved in votes
votes = []
weights = []

for index in range(len(cluster_number)):
  print('==================','working with cluster: ',index, '==================')
  sub_train_data = train_data[train_data[:,-1] == cluster_number[index]]
  #print('sub data size: ',len(sub_train_data))  #to see if the sub data is selected propoerly
  #print(sub_train_data[0:3,:],'\n')
  sub_train_data = sub_train_data[:,1:-1]  #index column and last cluster column 
  #print(sub_train_data[0:3,:],'\n')
  sub_train_x, sub_train_y = np.split(sub_train_data,[-1],axis=1)
  #display(sub_train_x[0:3,:])
  #display(sub_train_y[0:3,:])

  #converting the y to float
  sub_train_y = sub_train_y.astype('float64')

  sub_train_x = np.asarray(sub_train_x)
  sub_train_y = np.asarray(sub_train_y)
  sub_train_y = sub_train_y.ravel()

  model.fit(sub_train_x,sub_train_y) #train

  expected = y_test
  predicted = model.predict(x_test)
  #print('predicted: ',predicted[0:5])
  #print('expected: ',expected[0:5])
  acc_score = round(accuracy_score(expected, predicted, normalize=True),4)
  votes.append(predicted)
  weights.append(acc_score)
  
  print('\nAccuracy: ',acc_score,'\n')
  #print(metrics.classification_report(expected, predicted))
  
  #print('============================== DONE ================================\n')

votes=np.asarray(votes)
print(votes.shape)
for i in range(len(votes)):
  print('model ',i,': ',votes[i][0:10], ' weight: ',weights[i])

print('=================================')


###########################################################

kmeans.fit(x_test)

#print(kmeans.cluster_centers_)  


final_result = []
for index in range(len(y_test)):
  final_result.append(votes[kmeans.labels_[index]][index])
final_result = np.asarray(final_result)


print('suggested model: ',kmeans.labels_[0:10,],'\n')
print('expected       : ',expected[0:10])
print('final          : ',final_result[0:10])
accuracy = round(accuracy_score(expected, final_result, normalize=True),4)
print('\nAccuracy: ',accuracy,'\n')
print(metrics.classification_report(expected, final_result))

object data types:  9
after the conversion:  [2 3 0 0 0 0 0 2 2]
----------debug start----------
main data:  (12960, 9)
x:  (12960, 8)
y:  (12960, 1)
x_train:  (12895, 8)
x_test:  (65, 8)
y_train:  (12895,)
y_test:  (65,)
----------debug end----------
optimal no. of k =  5 

cluster labels:  [2 4 0 3 3] 

my train_data_size:  (12895, 11)
Cluster No:  [0 1 2 3 4]
Instance Count :  [2420 2578 2900 2583 2414]
================== working with cluster:  0 ==================

Accuracy:  0.6308 

================== working with cluster:  1 ==================

Accuracy:  0.7692 

================== working with cluster:  2 ==================

Accuracy:  0.8154 

================== working with cluster:  3 ==================

Accuracy:  0.7846 

================== working with cluster:  4 ==================

Accuracy:  0.6615 

(5, 65)
model  0 :  [0. 1. 0. 4. 0. 0. 1. 4. 4. 0.]  weight:  0.6308
model  1 :  [0. 3. 0. 1. 0. 0. 1. 1. 1. 0.]  weight:  0.7692
model  2 :  [0. 1. 0. 1. 0. 0. 1. 3. 3. 

Below attached is the basic Naive Bayesian Model for comparison

In [26]:
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

url='https://raw.githubusercontent.com/sak1b0/Thesis/master/allCSV/nursery.csv'

df=pd.read_csv(url,header=None)
#print(df.iloc[0,:])

#----------to handle nominal values---------

my_arr=[]

#track of column indices 
index=0
for item in df.dtypes:
  if(item=='object'):
    #if data type is nominal adding to the array
    my_arr.append(index)
  index=index+1
  
print('object data types: ',len(my_arr))

df=df.values

if(len(my_arr)>0):
  lbl=LabelEncoder()
  for item in my_arr:
    df[:,item] = lbl.fit_transform(df[:,item])
    
  print('after the conversion: ',df[0])    
#--------------------------------------------


df=np.asarray(df)

X, y = np.split(df,[-1],axis=1)
#issue here !!!!!!!!!!!!!!!!!!!!!!!!!!!!!
y=y.astype('float64')

x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=101)

#to check additional info set the value of debug to 1
debug=0

def debug_me():
  print('----------debug start----------')
  print('main data: ',df.shape)
  print('x: ',X.shape)
  print('y: ',y.shape)
  print('x_train: ',x_train.shape)
  print('x_test: ',x_test.shape)
  print('y_train: ',y_train.shape)
  print('y_test: ',y_test.shape)
  print('----------debug end----------')




# X=np.asarray(X)
# y=np.asarray(y)
# y=y.ravel() #vector to array

x_train=np.asarray(x_train)
x_test=np.asarray(x_test)
y_train=np.asarray(y_train)
y_test=np.asarray(y_test)
y_train=y_train.ravel()
y_test=y_test.ravel()


#model = MultinomialNB()
model = GaussianNB()
model.fit(x_train,y_train)
expected = y_test
predicted = model.predict(x_test)
accuracy = round(accuracy_score(expected, predicted, normalize=True),4)
print('\nAccuracy: ',accuracy,'\n')
print(metrics.classification_report(expected, predicted))
#print(metrics.confusion_matrix(expected, predicted))

object data types:  9
after the conversion:  [2 3 0 0 0 0 0 2 2]

Accuracy:  0.6469 

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      1289
         1.0       0.81      0.33      0.47      1273
         2.0       0.00      0.00      0.00         1
         3.0       0.83      0.57      0.68      1221
         4.0       0.08      1.00      0.16       104

    accuracy                           0.65      3888
   macro avg       0.54      0.58      0.46      3888
weighted avg       0.86      0.65      0.70      3888



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
